# CIC-Darknet2020 Dataset Cleaning

Here we load data from the [CIC-Darknet2020](https://www.unb.ca/cic/datasets/darknet2020.html) dataset remove any invalid values from the dataset and save the cleaned dataset to a new file.

First we import all relevant libraries, set a random seed, and print python and library versions for reproducability

In [16]:
import datetime, os, platform, pprint, sys
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

seed: int = 14

# set up pretty printer for easier data evaluation
pretty = pprint.PrettyPrinter(indent=4, width=30).pprint

# set up pandas display options
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

print(
    f'''
    Last Execution: {datetime.datetime.now()}
    python:\t{platform.python_version()}

    \tmatplotlib:\t{mpl.__version__}
    \tnumpy:\t\t{np.__version__}
    \tpandas:\t\t{pd.__version__}
    '''
)


    Last Execution: 2022-02-16 20:59:06.427550
    python:	3.7.10

    	matplotlib:	3.3.4
    	numpy:		1.20.3
    	pandas:		1.2.5
    


Next we prepare some helper functions to help process the data

In [17]:
def get_file_path(directory: str):
    '''
        Closure that will return a function. 
        Function will return the filepath to the directory given to the closure
    '''

    def func(file: str) -> str:
        return os.path.join(directory, file)

    return func



def load_data(filePath):
    '''
        Loads the Dataset from the given filepath and caches it for quick access in the future
        Function will only work when filepath is a .csv file
    '''

    # slice off the ./CSV/ from the filePath
    if filePath[0] == '.' and filePath[1] == '/':
        filePathClean: str = filePath[11::]
        pickleDump: str = f'./cache/{filePathClean}.pickle'
    else:
        pickleDump: str = f'./cache/{filePath}.pickle'
    
    print(f'Loading Dataset: {filePath}')
    print(f'\tTo Dataset Cache: {pickleDump}\n')


    # check if data already exists within cache
    if os.path.exists(pickleDump):
        df = pd.read_pickle(pickleDump)
        df['Label1'] = df['Label1'].str.lower()
        df.Label1.unique()    

    # if not, load data and clean it before caching it
    else:
        df = pd.read_csv(filePath, low_memory=True)
        df['Label1'] = df['Label1'].str.lower()
        df.Label1.unique()    
        df.to_pickle(pickleDump)
    
    return df



def features_with_bad_values(df: pd.DataFrame, datasetName: str) -> pd.DataFrame:
    '''
        Function will scan the dataframe for features with Inf, NaN, or Zero values.
        Returns a new dataframe describing the distribution of these values in the original dataframe
    '''

    # Inf and NaN values can take different forms so we screen for every one of them
    invalid_values: list = [ np.inf, np.nan, 'Infinity', 'inf', 'NaN', 'nan', 0 ]
    infs          : list = [ np.inf, 'Infinity', 'inf' ]
    NaNs          : list = [ np.nan, 'NaN', 'nan' ]

    # We will collect stats on the dataset, specifically how many instances of Infs, NaNs, and 0s are present.
    # using a dictionary that will be converted into a (3, 2+88) dataframe
    stats: dict = {
        'Dataset':[ datasetName, datasetName, datasetName ],
        'Value'  :['Inf', 'NaN', 'Zero']
    }

    i = 0
    for col in df.columns:
        
        i += 1
        feature = np.zeros(3)
        
        for value in invalid_values:
            if value in infs:
                j = 0
            elif value in NaNs:
                j = 1
            else:
                j = 2
            indexNames = df[df[col] == value].index
            if not indexNames.empty:
                feature[j] += len(indexNames)
                
        stats[col] = feature

    return pd.DataFrame(stats)



def clean_data(df: pd.DataFrame, prune: list) -> pd.DataFrame:
    '''
        Function will take a dataframe and remove the columns that match a value in prune 
        Inf values will also be removed from Flow Bytes/s and Flow Packets/s
        once appropriate rows and columns have been removed, we will return
        the dataframe with the appropriate values
    '''

    # remove the features in the prune list    
    for col in prune:
        if col in df.columns:
            df.drop(columns=[col], inplace=True)

    
    # drop missing values/NaN etc.
    df.dropna(inplace=True)

    
    # Search through dataframe for any Infinite or NaN values in various forms that were not picked up previously
    invalid_values: list = [
        np.inf, np.nan, 'Infinity', 'inf', 'NaN', 'nan'
    ]
    
    for col in df.columns:
        for value in invalid_values:
            indexNames = df[df[col] == value].index
            if not indexNames.empty:
                print(f'deleting {len(indexNames)} rows with Infinity in column {col}')
                df.drop(indexNames, inplace=True)

    return df


Before we do any processing on the data, we need to list out all their filepaths. If trying to reproduce the process carried out here, place files in the same location relative to the notebook.

In [18]:
# This code is used to scale to processing numerous datasets, even though we currently are only looking at one now
data_path_1: str = './original/'   
data_set_1: list = [
    'Darknet.csv',
]

data_set: list  = data_set_1
file_path_1      = get_file_path(data_path_1)
file_set: list   = list(map(file_path_1, data_set_1))
current_job: int = 0

Some more helper functions that process the data using the file and dataset information above

In [33]:
def examine_dataset(job_id: int) -> dict({'File': str, 'Dataset': pd.DataFrame, 'Feature_stats': pd.DataFrame, 'Data_composition': pd.DataFrame}):
    '''
        Function will return a dictionary containing dataframe of the job_id passed in as well as that dataframe's
        feature stats, data composition, and file name.
    '''

    job_id = job_id - 1  # adjusts for indexing while enumerating jobs from 1
    print(f'Dataset {job_id+1}/{len(data_set)}: We now look at {file_set[job_id]}\n\n')


    # Load the dataset
    df: pd.DataFrame = load_data(file_set[job_id])


    # print the data composition
    print(f'''
        File:\t\t\t\t{file_set[job_id]}  
        Job Number:\t\t\t{job_id+1}
        Shape:\t\t\t\t{df.shape}
        Samples:\t\t\t{df.shape[0]} 
        Features:\t\t\t{df.shape[1]}
    ''')


    # return the dataframe and the feature stats
    data_summary: dict =  {
        'File':             file_set[job_id],
        'Dataset':          df,
        'Feature_stats':    features_with_bad_values(df, file_set[job_id]), 
    }
    
    return data_summary


def package_data_for_inspection(df: pd.DataFrame) -> dict({'File': str, 'Dataset': pd.DataFrame, 'Feature_stats': pd.DataFrame, 'Data_composition': pd.DataFrame}):
    '''
        Function will return a dictionary containing dataframe passed in as well as that dataframe's feature stats.
    '''

    # print the data composition
    print(f'''
        Shape:\t\t\t\t{df.shape}
        Samples:\t\t\t{df.shape[0]} 
        Features:\t\t\t{df.shape[1]}
    ''')
    

    # return the dataframe and the feature stats
    data_summary: dict =  {
        'File':             '',
        'Dataset':          df,
        'Feature_stats':    features_with_bad_values(df, ''), 
    }
    
    return data_summary


def package_data_for_inspection_with_label(df: pd.DataFrame, label: str) -> dict({'File': str, 'Dataset': pd.DataFrame, 'Feature_stats': pd.DataFrame, 'Data_composition': pd.DataFrame}):
    '''
        Function will return a dictionary containing dataframe passed in as well as that dataframe's feature stats.
    '''

    # print the data composition
    print(f'''
        Shape:\t\t\t\t{df.shape}
        Samples:\t\t\t{df.shape[0]} 
        Features:\t\t\t{df.shape[1]}
    ''')
    

    # return the dataframe and the feature stats
    data_summary: dict =  {
        'File':             f'{label}',
        'Dataset':          df,
        'Feature_stats':    features_with_bad_values(df, f'{label}'), 
    }
    
    return data_summary



def check_infs(data_summary: dict) -> pd.DataFrame:
    '''
        Function will return a dataframe of features with a value of Inf.
    '''

    
    vals: pd.DataFrame = data_summary['Feature_stats']
    inf_df = vals[vals['Value'] == 'Inf'].T

    return inf_df[inf_df[0] != 0]



def check_nans(data_summary: dict) -> pd.DataFrame:
    '''
        Function will return a dataframe of features with a value of NaN.
    '''

    vals: pd.DataFrame = data_summary['Feature_stats']
    nan_df = vals[vals['Value'] == 'NaN'].T

    return nan_df[nan_df[1] != 0]



def check_zeros(data_summary: dict) -> pd.DataFrame:
    '''
        Function will return a dataframe of features with a value of 0.
    '''

    vals: pd.DataFrame = data_summary['Feature_stats']
    zero_df = vals[vals['Value'] == 'Zero'].T

    return zero_df[zero_df[2] != 0]



def check_zeros_over_threshold(data_summary: dict, threshold: int) -> pd.DataFrame:
    '''
        Function will return a dataframe of features with a value of 0.
    '''

    vals: pd.DataFrame = data_summary['Feature_stats']
    zero_df = vals[vals['Value'] == 'Zero'].T
    zero_df_bottom = zero_df[2:]

    return zero_df_bottom[zero_df_bottom[2] > threshold]



def check_zeros_over_threshold_percentage(data_summary: dict, threshold: float) -> pd.DataFrame:
    '''
        Function will return a dataframe of features with all features with
        a frequency of 0 values greater than the threshold
    '''

    vals: pd.DataFrame = data_summary['Feature_stats']
    size: int = data_summary['Dataset'].shape[0]
    zero_df = vals[vals['Value'] == 'Zero'].T
    zero_df_bottom = zero_df[2:]

    return zero_df_bottom[zero_df_bottom[2] > threshold*size]


def remove_infs_and_nans(data_summary: dict) -> pd.DataFrame:
    '''
        Function will return the dataset with all inf and nan values removed.
    '''

    df: pd.DataFrame = data_summary['Dataset']
    df = clean_data(df, [])

    return df



def create_new_prune_candidates(zeros_df: pd.DataFrame) -> list:
    '''
        Function creates a list of prune candidates from a dataframe of features with a high frequency of 0 values
    '''

    return list(zeros_df.T.columns)



def intersection_of_prune_candidates(pruneCandidates: list, newPruneCandidates: list) -> list:
    '''
        Function will return a list of features that are in both pruneCandidates and newPruneCandidates
    '''

    return list(set(pruneCandidates).intersection(newPruneCandidates))



def test_infs(data_summary: dict) -> bool:
    '''
        Function asserts the dataset has no inf values.
    '''
    vals: pd.DataFrame = data_summary['Feature_stats']
    inf_df = vals[vals['Value'] == 'Inf'].T

    assert inf_df[inf_df[0] != 0].shape[0] == 2, 'Dataset has inf values'
    

    return True



def test_nans(data_summary: dict) -> bool:
    '''
        Function asserts the dataset has no NaN values
    '''

    vals: pd.DataFrame = data_summary['Feature_stats']
    nan_df = vals[vals['Value'] == 'NaN'].T

    assert nan_df[nan_df[1] != 0].shape[0] == 2, 'Dataset has NaN values'


    return True


   

This gives us a set of file locations. Lets look at the set of files that we will be cleaning

In [20]:
print(f'We will be cleaning {len(file_set)} files:')
pretty(file_set)

We will be cleaning 1 files:
['./original/Darknet.csv']


## The Original CIC-Darknet2020 Dataset

In [21]:
dataset_1: dict = examine_dataset(1)
print(f"""
    Labels in the first layer:
{dataset_1['Dataset'].groupby('Label').size()}

    Labels in the second layer:
 {dataset_1['Dataset'].groupby('Label1').size()}
""")

Dataset 1/1: We now look at ./original/Darknet.csv


Loading Dataset: ./original/Darknet.csv
	To Dataset Cache: ./cache/Darknet.csv.pickle


        File:				./original/Darknet.csv  
        Job Number:			1
        Shape:				(141530, 85)
        Samples:			141530 
        Features:			85
    

    Labels in the first layer:
Label
Non-Tor    93356
NonVPN     23863
Tor         1392
VPN        22919
dtype: int64

    Labels in the second layer:
 Label1
audio-streaming    18064
browsing           32808
chat               11478
email               6145
file-transfer      11182
p2p                48520
video-streaming     9767
voip                3566
dtype: int64



### Data Inspection

Here we just check to see how many inf and nan values are in the dataset.


In [32]:
df = check_infs(dataset_1)
df.shape

(4, 1)

In [23]:
check_nans(dataset_1)

,1
Dataset,./original/Darknet.csv
Value,NaN


In [24]:
check_zeros_over_threshold(dataset_1, dataset_1['Dataset'].shape[0]-1)

,2
Bwd PSH Flags,141530.0
Fwd URG Flags,141530.0
Bwd URG Flags,141530.0
URG Flag Count,141530.0
CWE Flag Count,141530.0
ECE Flag Count,141530.0
Fwd Bytes/Bulk Avg,141530.0
Fwd Packet/Bulk Avg,141530.0
Fwd Bulk Rate Avg,141530.0
Bwd Bytes/Bulk Avg,141530.0


### Data Cleaning

Now we remove the infs and nan values and then verify that the dataset no longer contains any infs or nans.

In [25]:
dataset_2 = package_data_for_inspection(remove_infs_and_nans(dataset_1))

deleting 2 rows with Infinity in column Flow Bytes/s

        Shape:				(141481, 85)
        Samples:			141481 
        Features:			85
    


In [26]:
check_infs(dataset_2)

,0
Dataset,
Value,Inf


In [27]:
check_nans(dataset_2)

,1
Dataset,
Value,NaN


In [28]:
check_zeros_over_threshold(dataset_2, dataset_2['Dataset'].shape[0]-1)

,2
Bwd PSH Flags,141481.0
Fwd URG Flags,141481.0
Bwd URG Flags,141481.0
URG Flag Count,141481.0
CWE Flag Count,141481.0
ECE Flag Count,141481.0
Fwd Bytes/Bulk Avg,141481.0
Fwd Packet/Bulk Avg,141481.0
Fwd Bulk Rate Avg,141481.0
Bwd Bytes/Bulk Avg,141481.0


In [35]:
if(test_infs(dataset_2) and test_nans(dataset_2)):
    print('Dataset is clean')

Dataset is clean


### Saving the Cleaned Dataset

Now we save the cleaned dataset to a new file.

In [29]:
dataset_2['Dataset'].to_csv('./cleaned/Darknet_cleaned.csv', index=False)

In [30]:
print(f'Last Execution: {datetime.datetime.now()}')
assert False, 'Nothing is complete after this point'

Last Execution: 2022-02-16 20:59:20.803303


AssertionError: Nothing is complete after this point